In [ ]:
#월별,보험사별 실적
import pandas as pd
dir0='E:/글로벌백업(180329)/04.수수료작업/'

months=['201901','201902','201903','201904','201905','201906','201907','201908',
        '201909','201910','201911','201912',
        '202001','202002','202003','202004','202005','202006','202007','202008',
        '202009']
# months=['202009']
rows=['지급년월','계약종류']
cols=['보험사']

  
def sumMonth(df):
    생보신계약=(df.계약종류=='생보') & (df.지급구분=='신계약') & (df.수수료계!=0)
    장기신계약=(df.계약종류=='장기') & (df.지급구분=='신계약') & (df.수수료계!=0) & (df.상태=='초회') 
    생보유지=(df.계약종류=='생보') & (df.지급구분=='유지')
    장기유지=(df.계약종류=='장기') & (df.지급구분=='유지')
    수수료=(df.계약종류=='생보') | (df.계약종류=='장기')
    자동차=(df.계약종류=='자동차')
    일반=(df.계약종류=='일반')
    
    vals=['보험료']
    dd1=pd.pivot_table(df.loc[생보신계약|장기신계약], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)
    vals=['원수사성적']
    dd2=pd.pivot_table(df.loc[생보신계약|장기신계약], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)

    vals=['보험료']
    dd3=pd.pivot_table(df.loc[생보유지|장기유지], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)

    vals=['수수료계']
    dd4=pd.pivot_table(df.loc[수수료], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)

    vals=['보험료']
    dd5=pd.pivot_table(df.loc[자동차], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)
    dd6=pd.pivot_table(df.loc[일반], values=vals,
            index=rows,
            columns=cols,
            fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=1,ascending=False)
    dt=pd.concat([dd1.stack(level=0),dd2.stack(level=0),dd3.stack(level=0).rename(index={'보험료': '유지'}),dd4.stack(level=0),dd5.stack(level=0),dd6.stack(level=0)])
    
    del dd1,dd2,dd3,dd4,dd5,dd6
    return dt.fillna(0)

def delCol(dfd):
    dfd=dfd.loc[:,(dfd.sum(axis=0) != 0)]
    dfd.loc[:,'합계']=dfd.sum(axis=1)
    dfd=dfd.fillna(0).reset_index()
    return dfd

print('start')    
dfAll=pd.DataFrame()
for ym in months:
    dirN=dir0+ym+'수수료/'
    fileN=ym[0:4]+'-'+ym[4:6]+' 수입원장(KJ).xlsx'
#     fileN='test원장.xlsx'
    print('reading ',fileN)
    df=pd.read_excel(dirN+fileN,sheet_name='전체')
    dfT=sumMonth(df)
    dfAll=pd.concat([dfAll,dfT])

print('df end...')
df_nl=delCol(dfAll.loc[(slice(None),['장기','자동차','일반'],slice(None)),:])
df_li=delCol(dfAll.loc[(slice(None),['생보'],slice(None)),:])

wrt_file="E:/글로벌백업(180329)/승격총괄/보험사별실적.xlsx"
with pd.ExcelWriter(wrt_file) as writer:
    df_nl.to_excel(writer, sheet_name='손보실적',index=False),
    df_li.to_excel(writer, sheet_name='생보실적',index=False),
        
print('completed')

In [ ]:
import pandas as pd
#설계사별 건수 및 실적
dir0='E:/글로벌백업(180329)/04.수수료작업/'
# months=['202004','202005','202006','202007','202008','202009']
months=['202009']
vals=['증권번호']
fcrows=['지급년월','지점','사원번호','사원명']
fcols=['계약종류']

dfFCAll=pd.DataFrame()

def sumFC(df):
    생보신계약=(df.계약종류=='생보') & (df.지급구분=='신계약') & (df.수수료계!=0)
    장기신계약=(df.계약종류=='장기') & (df.지급구분=='신계약') & (df.수수료계!=0) & (df.상태=='초회') 
    자동차=(df.계약종류=='자동차')
    일반=(df.계약종류=='일반')
    
    vals=['증권번호']
    dd1=pd.pivot_table(df.loc[생보신계약|장기신계약], values=vals,
                index=fcrows,
                columns=fcols,
                fill_value=0, aggfunc=pd.Series.nunique,dropna=True, ).sort_index(axis=0,ascending=False)
    dd1.rename(columns={'장기': '장기건수','생보': '생보건수'},inplace=True)

    vals=['보험료']
    dd2=pd.pivot_table(df.loc[생보신계약|장기신계약|자동차|일반], values=vals,
                index=fcrows,
                columns=fcols,
                fill_value=0, aggfunc=sum,dropna=True, ).sort_index(axis=0,ascending=False)
    dt=pd.concat([dd1,dd2],axis=1)
    del dd1,dd2
    return dt.fillna(0)

print('start')
for ym in months:
    dirN=dir0+ym+'수수료/'
    fileN=ym[0:4]+'-'+ym[4:6]+' 수입원장(KJ).xlsx'
#     fileN='test원장.xlsx'
    print('reading ',fileN)
    df=pd.read_excel(dirN+fileN,sheet_name='전체')
    df['지점'].fillna('엠피메트로', inplace=True)  
    dfT=sumFC(df)
    dfFCAll=pd.concat([dfFCAll,dfT])

dfFC=dfFCAll.groupby(['지점','사원번호','사원명']).sum()
dfFC=round(dfFC/len(months),1)
dfFC=dfFC.fillna(0).reset_index()
# wrt_file="E:/글로벌백업(180329)/승격총괄/설계사별실적.xlsx"
# with pd.ExcelWriter(wrt_file) as writer:
#       dfFC.to_excel(writer, sheet_name='실적')
        
print('completed')

In [ ]:
dfFC=dt
dfFC=round(dfFC/6,0)
dfFC=dfFC.fillna(0).reset_index()
wrt_file="E:/글로벌백업(180329)/승격총괄/bb.xlsx"
with pd.ExcelWriter(wrt_file) as writer:
      dfFC.to_excel(writer, sheet_name='실적')